## Download the data

In [1]:
#!pip install tensorflow
#!pip install tensorflow_hub
#!pip install tensorflow_text
#!pip install keras


import os.path
if not os.path.isfile('data/vmware_ir_content.csv'):
    !pip install kaggle
    !kaggle competitions download -c vmware-zero-shot-information-retrieval
    !mkdir -p data/
    !unzip -o vmware-zero-shot-information-retrieval.zip
    !mv *.csv data/
    
import tensorflow_text
import tensorflow_hub as hub
import tensorflow as tf
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

import numpy as np

import pandas as pd
queries = pd.read_csv('data/test.csv')

# Created with 'extract_use.py' script
queries = pd.read_csv("data/test.csv")
corpus = pd.read_pickle('data/vmware_ir_content_parsed.pkl')

In [2]:
from ltr.client import ElasticClient
client=ElasticClient()

mapping = {
  "properties": {
    "raw_text_use": {
        "type": "dense_vector",
        "dims": 512
    }
  }
}

def process_use(doc_source):    
    doc_source['raw_text_use'] = use(doc_source['raw_text']).numpy().tolist()
    return doc_source

client.enrich(index='vmware', enrich_fn=process_use, mapping=mapping, version=1)

RequestError: RequestError(400, 'illegal_argument_exception', 'mapper [raw_text_use] cannot be changed from type [float] to [dense_vector]')

## Title - body pair

Intuition:

* titles are like queries
* title,body label a positive example
* the body of the least similar title in an embedding space, is a negative example
* Task: use title_vect,body_vect to classify positive from negative examples in a supervised setting
*       during inference, use the query instead of a title to score each body as going with on not going with the title/query

In [17]:
def identify_question_in_title(row):
    
    title = row['titleTag']
    if isinstance(title, str) and '?' in title:
        true_question = ''
        base_title = title.split('|')[0]
        if '-' in base_title:
            for part in base_title.split('-'):
                if '?' in part:
                    true_question = part
                    break
        else:
            true_question = base_title
        true_question = true_question.strip()
        if len(true_question) > 17 and true_question.endswith('?'):
            return true_question
    return None
    
corpus['question'] = corpus.apply(identify_question_in_title, axis=1)
corpus

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name,question
0,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,ForgeRock Service Broker for VMware Tanzu | So...,authenticity_token,L4OfIONAOom_mTA9ShXlItMJGAUxY1nChie5y0hlPHKS0s...,ForgeRock Service Broker for VMware Tanzu | So...,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,What is DevSecOps? | VMware Tanzu,authenticity_token,5Incq1fozvzvWNsEzyt_YbeY-BEvHKmFW8WdNjxgSEi5m_...,What is DevSecOps?,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,What is DevSecOps?
2,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Dynatrace for VMware Tanzu | Solutions Hub | V...,authenticity_token,VqAn3GNy9pSHWoDrlxi_mxbAM6XrBFTcIDQeYUxvWCehtX...,Dynatrace for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Wells Fargo | Customers | VMware Tanzu,authenticity_token,1zYYUL38ZpMZ8IibiCBQXIUsYCQHvg-vccZWYXUtRy43sx...,Wells Fargo | Customers,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Hazelcast IMDG Enterprise for VMware Tanzu | S...,authenticity_token,5iPbM3ys6YlBwLP3FrdaDofy6AUo5ZhftBmNqOLN71Zm9S...,Hazelcast IMDG Enterprise for VMware Tanzu | S...,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
323959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
323960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
323961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [18]:
question_use = corpus[~corpus['question'].isna()]['question'].apply(use)

positives = pd.DataFrame()
positives['question_use'] = question_use
positives['raw_text_use'] = raw_text_use
positives['question'] = corpus[~corpus['question'].isna()]['question']
positives['raw_text'] = corpus[~corpus['question'].isna()]['raw_text']
positives['raw_text_use'] = corpus[~corpus['question'].isna()]['raw_text_use']


positives['label'] = 1.0

positives

,question_use,raw_text_use,question,raw_text,label
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,1.0
34,"((tf.Tensor(0.055269662, shape=(), dtype=float...","((tf.Tensor(-0.053562075, shape=(), dtype=floa...",What is Serverless?,What is Serverless? | VMware Tanzu\nTech Insig...,1.0
51,"((tf.Tensor(0.07775386, shape=(), dtype=float3...","((tf.Tensor(-0.053171247, shape=(), dtype=floa...",What is Data Science?,What is Data Science? | VMware Tanzu\nTech Ins...,1.0
78,"((tf.Tensor(0.008782897, shape=(), dtype=float...","((tf.Tensor(-0.05010325, shape=(), dtype=float...",What are Cloud Native Applications?,What are Cloud Native Applications? | VMware T...,1.0
102,"((tf.Tensor(0.04013639, shape=(), dtype=float3...","((tf.Tensor(-0.053587, shape=(), dtype=float32...",What is Digital Transformation?,What is Digital Transformation? | VMware Tanzu...,1.0
...,...,...,...,...,...
65215,"((tf.Tensor(0.046280254, shape=(), dtype=float...","((tf.Tensor(-0.04664653, shape=(), dtype=float...",What is Cloud Management Platform?,What is Cloud Management Platform? | VMware\nS...,1.0
65261,"((tf.Tensor(0.030358074, shape=(), dtype=float...","((tf.Tensor(-0.050994672, shape=(), dtype=floa...",What is VMware vSAN?,What is VMware vSAN? | Storage Virtualization ...,1.0
65272,"((tf.Tensor(-0.013038688, shape=(), dtype=floa...","((tf.Tensor(-0.04903052, shape=(), dtype=float...",What is vSphere Replication?,What is vSphere Replication? | Disaster Recove...,1.0
65294,"((tf.Tensor(-0.030024868, shape=(), dtype=floa...","((tf.Tensor(-0.05268035, shape=(), dtype=float...",What is a vSphere Hypervisor?,What is a vSphere Hypervisor? | Free Hyperviso...,1.0


In [19]:
def remove_question_from_text(row):
    question = row['question']
    return row['raw_text'].replace(question, '')

positives['raw_text_cleaned'] = positives.apply(remove_question_from_text, axis=1)
positives['raw_text_cleaned_use'] = positives['raw_text_cleaned'].apply(use)
positives

,question_use,raw_text_use,question,raw_text,label,raw_text_cleaned,raw_text_cleaned_use
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.050687112, shape=(), dtype=floa..."
34,"((tf.Tensor(0.055269662, shape=(), dtype=float...","((tf.Tensor(-0.053562075, shape=(), dtype=floa...",What is Serverless?,What is Serverless? | VMware Tanzu\nTech Insig...,1.0,| VMware Tanzu\nTech Insights5 Minute Read\nS...,"((tf.Tensor(-0.053588558, shape=(), dtype=floa..."
51,"((tf.Tensor(0.07775386, shape=(), dtype=float3...","((tf.Tensor(-0.053171247, shape=(), dtype=floa...",What is Data Science?,What is Data Science? | VMware Tanzu\nTech Ins...,1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(-0.053212874, shape=(), dtype=floa..."
78,"((tf.Tensor(0.008782897, shape=(), dtype=float...","((tf.Tensor(-0.05010325, shape=(), dtype=float...",What are Cloud Native Applications?,What are Cloud Native Applications? | VMware T...,1.0,| VMware Tanzu\nTech InsightsCloud native app...,"((tf.Tensor(-0.05014698, shape=(), dtype=float..."
102,"((tf.Tensor(0.04013639, shape=(), dtype=float3...","((tf.Tensor(-0.053587, shape=(), dtype=float32...",What is Digital Transformation?,What is Digital Transformation? | VMware Tanzu...,1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(-0.05371097, shape=(), dtype=float..."
...,...,...,...,...,...,...,...
65215,"((tf.Tensor(0.046280254, shape=(), dtype=float...","((tf.Tensor(-0.04664653, shape=(), dtype=float...",What is Cloud Management Platform?,What is Cloud Management Platform? | VMware\nS...,1.0,| VMware\nSolutions by Category\nVirtualizati...,"((tf.Tensor(-0.047710218, shape=(), dtype=floa..."
65261,"((tf.Tensor(0.030358074, shape=(), dtype=float...","((tf.Tensor(-0.050994672, shape=(), dtype=floa...",What is VMware vSAN?,What is VMware vSAN? | Storage Virtualization ...,1.0,| Storage Virtualization Software | VMware\nP...,"((tf.Tensor(-0.05098715, shape=(), dtype=float..."
65272,"((tf.Tensor(-0.013038688, shape=(), dtype=floa...","((tf.Tensor(-0.04903052, shape=(), dtype=float...",What is vSphere Replication?,What is vSphere Replication? | Disaster Recove...,1.0,| Disaster Recovery in VM | VMware\nProducts\...,"((tf.Tensor(-0.04908049, shape=(), dtype=float..."
65294,"((tf.Tensor(-0.030024868, shape=(), dtype=floa...","((tf.Tensor(-0.05268035, shape=(), dtype=float...",What is a vSphere Hypervisor?,What is a vSphere Hypervisor? | Free Hyperviso...,1.0,| Free Hypervisor | VMware\nProducts\nvSphere...,"((tf.Tensor(-0.052931365, shape=(), dtype=floa..."


211294    ((tf.Tensor(-0.04051689, shape=(), dtype=float...
Name: raw_text_use, dtype: object

In [226]:
negative_rounds = 1

all_negs = []

for _ in range(0, negative_rounds):
    negatives = positives.copy()
    negatives['raw_text_cleaned_use'] = negatives.sample(frac=1.0)['raw_text_use'].values
    negatives['label'] = -1.0
    all_negs.append(negatives)
    
for _ in range(0, negative_rounds):
    negatives = positives.copy()
    negatives['raw_text_cleaned_use'] = corpus.sample(len(positives))['raw_text_use'].values
    negatives['label'] = -1.0
    all_negs.append(negatives)

In [227]:
training_set = pd.concat(all_negs + [positives])
training_set

,question_use,raw_text_use,question,raw_text,label,raw_text_cleaned,raw_text_cleaned_use
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.053356435, shape=(), dtype=floa..."
34,"((tf.Tensor(0.055269662, shape=(), dtype=float...","((tf.Tensor(-0.053562075, shape=(), dtype=floa...",What is Serverless?,What is Serverless? | VMware Tanzu\nTech Insig...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nS...,"((tf.Tensor(-0.051827386, shape=(), dtype=floa..."
51,"((tf.Tensor(0.07775386, shape=(), dtype=float3...","((tf.Tensor(-0.053171247, shape=(), dtype=floa...",What is Data Science?,What is Data Science? | VMware Tanzu\nTech Ins...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(0.035800535, shape=(), dtype=float..."
78,"((tf.Tensor(0.008782897, shape=(), dtype=float...","((tf.Tensor(-0.05010325, shape=(), dtype=float...",What are Cloud Native Applications?,What are Cloud Native Applications? | VMware T...,-1.0,| VMware Tanzu\nTech InsightsCloud native app...,"((tf.Tensor(-0.046390884, shape=(), dtype=floa..."
102,"((tf.Tensor(0.04013639, shape=(), dtype=float3...","((tf.Tensor(-0.053587, shape=(), dtype=float32...",What is Digital Transformation?,What is Digital Transformation? | VMware Tanzu...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(-0.05094526, shape=(), dtype=float..."
...,...,...,...,...,...,...,...
65215,"((tf.Tensor(0.046280254, shape=(), dtype=float...","((tf.Tensor(-0.04664653, shape=(), dtype=float...",What is Cloud Management Platform?,What is Cloud Management Platform? | VMware\nS...,1.0,| VMware\nSolutions by Category\nVirtualizati...,"((tf.Tensor(-0.047710218, shape=(), dtype=floa..."
65261,"((tf.Tensor(0.030358074, shape=(), dtype=float...","((tf.Tensor(-0.050994672, shape=(), dtype=floa...",What is VMware vSAN?,What is VMware vSAN? | Storage Virtualization ...,1.0,| Storage Virtualization Software | VMware\nP...,"((tf.Tensor(-0.05098715, shape=(), dtype=float..."
65272,"((tf.Tensor(-0.013038688, shape=(), dtype=floa...","((tf.Tensor(-0.04903052, shape=(), dtype=float...",What is vSphere Replication?,What is vSphere Replication? | Disaster Recove...,1.0,| Disaster Recovery in VM | VMware\nProducts\...,"((tf.Tensor(-0.04908049, shape=(), dtype=float..."
65294,"((tf.Tensor(-0.030024868, shape=(), dtype=floa...","((tf.Tensor(-0.05268035, shape=(), dtype=float...",What is a vSphere Hypervisor?,What is a vSphere Hypervisor? | Free Hyperviso...,1.0,| Free Hypervisor | VMware\nProducts\nvSphere...,"((tf.Tensor(-0.052931365, shape=(), dtype=floa..."


## Confirm training set behaves as expected

- All question embeddings should be identical per question
- Raw text embeddings should NOT

In [228]:
dev_sec_ops = training_set[training_set['question'] == 'What is DevSecOps?']
assert (dev_sec_ops['question_use'] == dev_sec_ops.iloc[0]['question_use']).all()
dev_sec_ops

,question_use,raw_text_use,question,raw_text,label,raw_text_cleaned,raw_text_cleaned_use
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.053356435, shape=(), dtype=floa..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.044252545, shape=(), dtype=floa..."
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.050687112, shape=(), dtype=floa..."


In [229]:
def format_X(training_set):
    raw_text_use = training_set['raw_text_cleaned_use'].numpy()[0]
    question_use = training_set['question_use'].numpy()[0]
    concated = np.concatenate([question_use, raw_text_use])
    return concated
    #return np.concatenate(training_set['question_use'].numpy()[0],
    #                      training_set['raw_text_use'].numpy()[0])

training_set['X'] = training_set.apply(format_X, axis=1)
training_set

,question_use,raw_text_use,question,raw_text,label,raw_text_cleaned,raw_text_cleaned_use,X
1,"((tf.Tensor(0.025211554, shape=(), dtype=float...","((tf.Tensor(-0.05063145, shape=(), dtype=float...",What is DevSecOps?,What is DevSecOps? | VMware Tanzu\nTech Insigh...,-1.0,| VMware Tanzu\nTech InsightsDevSecOps: Integ...,"((tf.Tensor(-0.053356435, shape=(), dtype=floa...","[0.025211554, 0.028694073, 0.02947252, 0.05354..."
34,"((tf.Tensor(0.055269662, shape=(), dtype=float...","((tf.Tensor(-0.053562075, shape=(), dtype=floa...",What is Serverless?,What is Serverless? | VMware Tanzu\nTech Insig...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nS...,"((tf.Tensor(-0.051827386, shape=(), dtype=floa...","[0.055269662, 0.042888615, 0.03670096, 0.03087..."
51,"((tf.Tensor(0.07775386, shape=(), dtype=float3...","((tf.Tensor(-0.053171247, shape=(), dtype=floa...",What is Data Science?,What is Data Science? | VMware Tanzu\nTech Ins...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(0.035800535, shape=(), dtype=float...","[0.07775386, 0.044761658, 0.054449685, 0.03132..."
78,"((tf.Tensor(0.008782897, shape=(), dtype=float...","((tf.Tensor(-0.05010325, shape=(), dtype=float...",What are Cloud Native Applications?,What are Cloud Native Applications? | VMware T...,-1.0,| VMware Tanzu\nTech InsightsCloud native app...,"((tf.Tensor(-0.046390884, shape=(), dtype=floa...","[0.008782897, 0.070924744, -0.0039736154, -0.0..."
102,"((tf.Tensor(0.04013639, shape=(), dtype=float3...","((tf.Tensor(-0.053587, shape=(), dtype=float32...",What is Digital Transformation?,What is Digital Transformation? | VMware Tanzu...,-1.0,| VMware Tanzu\nTech Insights5 Minute Read\nD...,"((tf.Tensor(-0.05094526, shape=(), dtype=float...","[0.04013639, 0.0390353, 0.04971546, 0.03647691..."
...,...,...,...,...,...,...,...,...
65215,"((tf.Tensor(0.046280254, shape=(), dtype=float...","((tf.Tensor(-0.04664653, shape=(), dtype=float...",What is Cloud Management Platform?,What is Cloud Management Platform? | VMware\nS...,1.0,| VMware\nSolutions by Category\nVirtualizati...,"((tf.Tensor(-0.047710218, shape=(), dtype=floa...","[0.046280254, 0.031247193, 0.015304046, -0.073..."
65261,"((tf.Tensor(0.030358074, shape=(), dtype=float...","((tf.Tensor(-0.050994672, shape=(), dtype=floa...",What is VMware vSAN?,What is VMware vSAN? | Storage Virtualization ...,1.0,| Storage Virtualization Software | VMware\nP...,"((tf.Tensor(-0.05098715, shape=(), dtype=float...","[0.030358074, 0.018951591, 0.06789018, -0.0155..."
65272,"((tf.Tensor(-0.013038688, shape=(), dtype=floa...","((tf.Tensor(-0.04903052, shape=(), dtype=float...",What is vSphere Replication?,What is vSphere Replication? | Disaster Recove...,1.0,| Disaster Recovery in VM | VMware\nProducts\...,"((tf.Tensor(-0.04908049, shape=(), dtype=float...","[-0.013038688, 0.02732803, 0.06733354, -0.0581..."
65294,"((tf.Tensor(-0.030024868, shape=(), dtype=floa...","((tf.Tensor(-0.05268035, shape=(), dtype=float...",What is a vSphere Hypervisor?,What is a vSphere Hypervisor? | Free Hyperviso...,1.0,| Free Hypervisor | VMware\nProducts\nvSphere...,"((tf.Tensor(-0.052931365, shape=(), dtype=floa...","[-0.030024868, 0.017225867, 0.07627318, -0.075..."


## Confirm embeddings work as expected

For a given question/title - our question embeddings should still be first 512, and be identical.

In [230]:
dev_sec_ops = training_set[training_set['question'] == 'What is DevSecOps?']
assert (dev_sec_ops['question_use'] == dev_sec_ops.iloc[0]['question_use']).all()

def first_of_X(X):
    """First 512 should be identical"""
    sliced = X[:512]
    assert sliced.size == 512, f"Sliced size is {sliced.size}"
    return sliced

should_be_same = dev_sec_ops['X'].apply(first_of_X)
first_value = should_be_same.values[0]
# should be a cleaner way to do this with pandas, why doesn't the assert above work? Check later
for value in should_be_same.values:
    assert (first_value == value).all()

## Evaluate a model with cross validation

Train a binary classifier with Keras using binary classification, use cross validation to find the best model

In [231]:
!pip install tqdm

In [233]:
import tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

def build_model():
    model = Sequential()
    model.add(Dense(64, input_dim=1024, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

kfold = StratifiedKFold(n_splits=8, shuffle=True)
estimator = KerasClassifier(build_fn=build_model, epochs=500, batch_size=100, verbose=1)

results = cross_val_score(estimator, 
                          np.stack(training_set['X'].to_numpy()),
                          training_set['label'],
                          cv=kfold)

print(results, results.mean(), results.std())

Epoch 1/500


<ipython-input-233-7b1d23fe0543>:15: DeprecationWarning:

KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.



24/24 [==============================] - 0s 1ms/step - loss: 0.6296 - accuracy: 0.6664
Epoch 2/500
24/24 [==============================] - 0s 1ms/step - loss: 0.6037 - accuracy: 0.6664
Epoch 3/500
24/24 [==============================] - 0s 2ms/step - loss: 0.5858 - accuracy: 0.6664
Epoch 4/500
24/24 [==============================] - 0s 1ms/step - loss: 0.5742 - accuracy: 0.6664
Epoch 5/500
24/24 [==============================] - 0s 1ms/step - loss: 0.5641 - accuracy: 0.6707
Epoch 6/500
24/24 [==============================] - 0s 1ms/step - loss: 0.5562 - accuracy: 0.6699
Epoch 7/500
24/24 [==============================] - 0s 1ms/step - loss: 0.5493 - accuracy: 0.6720
Epoch 8/500
24/24 [==============================] - 0s 1ms/step - loss: 0.5433 - accuracy: 0.6681
Epoch 9/500
24/24 [==============================] - 0s 2ms/step - loss: 0.5406 - accuracy: 0.6742
Epoch 10/500
24/24 [==============================] - 0s 1ms/step - loss: 0.5355 - accuracy: 0.6668
Epoch 11/500
24/24 [=

24/24 [==============================] - 0s 1ms/step - loss: 0.4800 - accuracy: 0.7168
Epoch 84/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4800 - accuracy: 0.7142
Epoch 85/500
24/24 [==============================] - 0s 2ms/step - loss: 0.4809 - accuracy: 0.7163
Epoch 86/500
24/24 [==============================] - 0s 2ms/step - loss: 0.4805 - accuracy: 0.7120
Epoch 87/500
24/24 [==============================] - 0s 2ms/step - loss: 0.4816 - accuracy: 0.7137
Epoch 88/500
24/24 [==============================] - 0s 2ms/step - loss: 0.4778 - accuracy: 0.7198
Epoch 89/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4792 - accuracy: 0.7189
Epoch 90/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4777 - accuracy: 0.7228
Epoch 91/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4774 - accuracy: 0.7159
Epoch 92/500
24/24 [==============================] - 0s 2ms/step - loss: 0.4765 - accuracy: 0.7224
Epoch 93/500


24/24 [==============================] - 0s 1ms/step - loss: 0.4360 - accuracy: 0.7715
Epoch 165/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4345 - accuracy: 0.7702
Epoch 166/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4333 - accuracy: 0.7711
Epoch 167/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4326 - accuracy: 0.7702
Epoch 168/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4322 - accuracy: 0.7732
Epoch 169/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4305 - accuracy: 0.7758
Epoch 170/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4288 - accuracy: 0.7815
Epoch 171/500
24/24 [==============================] - 0s 2ms/step - loss: 0.4273 - accuracy: 0.7785
Epoch 172/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4269 - accuracy: 0.7758
Epoch 173/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4263 - accuracy: 0.7798
Epoc

...

24/24 [==============================] - 0s 1ms/step - loss: 0.4367 - accuracy: 0.7620
Epoch 147/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4357 - accuracy: 0.7642
Epoch 148/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4373 - accuracy: 0.7590
Epoch 149/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4335 - accuracy: 0.7668
Epoch 150/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4348 - accuracy: 0.7547
Epoch 151/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4332 - accuracy: 0.7625
Epoch 152/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4306 - accuracy: 0.7673
Epoch 153/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4299 - accuracy: 0.7677
Epoch 154/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4277 - accuracy: 0.7668
Epoch 155/500
24/24 [==============================] - 0s 1ms/step - loss: 0.4270 - accuracy: 0.7660
Epoc

24/24 [==============================] - 0s 1ms/step - loss: 0.3325 - accuracy: 0.8472
Epoch 228/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3317 - accuracy: 0.8485
Epoch 229/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3323 - accuracy: 0.8493
Epoch 230/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3304 - accuracy: 0.8545
Epoch 231/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3314 - accuracy: 0.8489
Epoch 232/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3309 - accuracy: 0.8450
Epoch 233/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3347 - accuracy: 0.8333
Epoch 234/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3275 - accuracy: 0.8550
Epoch 235/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3260 - accuracy: 0.8584
Epoch 236/500
24/24 [==============================] - 0s 1ms/step - loss: 0.3249 - accuracy: 0.8558
Epoc

24/24 [==============================] - 0s 1ms/step - loss: 0.2553 - accuracy: 0.8975
Epoch 309/500
24/24 [==============================] - 0s 2ms/step - loss: 0.2602 - accuracy: 0.8919
Epoch 310/500
24/24 [==============================] - 0s 1ms/step - loss: 0.2522 - accuracy: 0.9027
Epoch 311/500
24/24 [==============================] - 0s 2ms/step - loss: 0.2543 - accuracy: 0.9001
Epoch 312/500
24/24 [==============================] - 0s 2ms/step - loss: 0.2508 - accuracy: 0.9049
Epoch 313/500
24/24 [==============================] - 0s 1ms/step - loss: 0.2521 - accuracy: 0.9027
Epoch 314/500
24/24 [==============================] - 0s 2ms/step - loss: 0.2492 - accuracy: 0.9019
Epoch 315/500
24/24 [==============================] - 0s 2ms/step - loss: 0.2497 - accuracy: 0.9019
Epoch 316/500
24/24 [==============================] - 0s 1ms/step - loss: 0.2469 - accuracy: 0.9066
Epoch 317/500
24/24 [==============================] - 0s 1ms/step - loss: 0.2496 - accuracy: 0.8988
Epoc

24/24 [==============================] - 0s 1ms/step - loss: 0.1836 - accuracy: 0.9396
Epoch 390/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1818 - accuracy: 0.9431
Epoch 391/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1807 - accuracy: 0.9440
Epoch 392/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1800 - accuracy: 0.9436
Epoch 393/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1799 - accuracy: 0.9440
Epoch 394/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1788 - accuracy: 0.9422
Epoch 395/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1780 - accuracy: 0.9436
Epoch 396/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1790 - accuracy: 0.9444
Epoch 397/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1791 - accuracy: 0.9444
Epoch 398/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1757 - accuracy: 0.9475
Epoc

24/24 [==============================] - 0s 1ms/step - loss: 0.1220 - accuracy: 0.9766
Epoch 471/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1263 - accuracy: 0.9683
Epoch 472/500
24/24 [==============================] - 0s 2ms/step - loss: 0.1210 - accuracy: 0.9748
Epoch 473/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1202 - accuracy: 0.9774
Epoch 474/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1217 - accuracy: 0.9731
Epoch 475/500
24/24 [==============================] - 0s 2ms/step - loss: 0.1203 - accuracy: 0.9731
Epoch 476/500
24/24 [==============================] - 0s 2ms/step - loss: 0.1212 - accuracy: 0.9770
Epoch 477/500
24/24 [==============================] - 0s 2ms/step - loss: 0.1186 - accuracy: 0.9792
Epoch 478/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1200 - accuracy: 0.9761
Epoch 479/500
24/24 [==============================] - 0s 1ms/step - loss: 0.1164 - accuracy: 0.9774
Epoc

In [234]:
len(training_set)//10

263

In [235]:
estimator.fit( np.stack(training_set['X'].to_numpy()),
               training_set['label'])

Epoch 1/500
27/27 [==============================] - 0s 1ms/step - loss: 0.6223 - accuracy: 0.6667
Epoch 2/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5938 - accuracy: 0.6667
Epoch 3/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5752 - accuracy: 0.6644
Epoch 4/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5606 - accuracy: 0.6655
Epoch 5/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5510 - accuracy: 0.6629
Epoch 6/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5439 - accuracy: 0.6537
Epoch 7/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5380 - accuracy: 0.6526
Epoch 8/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5343 - accuracy: 0.6556
Epoch 9/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5311 - accuracy: 0.6534
Epoch 10/500
27/27 [==============================] - 0s 1ms/step - loss: 0.5279 - accuracy: 0.6465
Epoch 11/

27/27 [==============================] - 0s 1ms/step - loss: 0.4772 - accuracy: 0.7111
Epoch 84/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4741 - accuracy: 0.7168
Epoch 85/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4735 - accuracy: 0.7191
Epoch 86/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4725 - accuracy: 0.7199
Epoch 87/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4720 - accuracy: 0.7256
Epoch 88/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4719 - accuracy: 0.7195
Epoch 89/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4707 - accuracy: 0.7244
Epoch 90/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4689 - accuracy: 0.7294
Epoch 91/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4688 - accuracy: 0.7282
Epoch 92/500
27/27 [==============================] - 0s 1ms/step - loss: 0.4677 - accuracy: 0.7267
Epoch 93/500


27/27 [==============================] - 0s 1ms/step - loss: 0.3470 - accuracy: 0.8381
Epoch 165/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3462 - accuracy: 0.8407
Epoch 166/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3429 - accuracy: 0.8426
Epoch 167/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3427 - accuracy: 0.8434
Epoch 168/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3402 - accuracy: 0.8453
Epoch 169/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3399 - accuracy: 0.8430
Epoch 170/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3384 - accuracy: 0.8423
Epoch 171/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3361 - accuracy: 0.8445
Epoch 172/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3346 - accuracy: 0.8510
Epoch 173/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3328 - accuracy: 0.8472
Epoc

27/27 [==============================] - 0s 1ms/step - loss: 0.2350 - accuracy: 0.9164
Epoch 246/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2334 - accuracy: 0.9164
Epoch 247/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2326 - accuracy: 0.9175
Epoch 248/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2313 - accuracy: 0.9168
Epoch 249/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2306 - accuracy: 0.9164
Epoch 250/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2288 - accuracy: 0.9152
Epoch 251/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2270 - accuracy: 0.9202
Epoch 252/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2261 - accuracy: 0.9187
Epoch 253/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2250 - accuracy: 0.9202
Epoch 254/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2241 - accuracy: 0.9209
Epoc

27/27 [==============================] - 0s 1ms/step - loss: 0.1395 - accuracy: 0.9681
Epoch 327/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1378 - accuracy: 0.9688
Epoch 328/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1364 - accuracy: 0.9681
Epoch 329/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1354 - accuracy: 0.9688
Epoch 330/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1351 - accuracy: 0.9711
Epoch 331/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1335 - accuracy: 0.9696
Epoch 332/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1321 - accuracy: 0.9696
Epoch 333/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1311 - accuracy: 0.9742
Epoch 334/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1308 - accuracy: 0.9704
Epoch 335/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1292 - accuracy: 0.9719
Epoc

27/27 [==============================] - 0s 1ms/step - loss: 0.0676 - accuracy: 0.9966
Epoch 408/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0668 - accuracy: 0.9966
Epoch 409/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0665 - accuracy: 0.9973
Epoch 410/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0660 - accuracy: 0.9977
Epoch 411/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0649 - accuracy: 0.9973
Epoch 412/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0643 - accuracy: 0.9977
Epoch 413/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0640 - accuracy: 0.9981
Epoch 414/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0631 - accuracy: 0.9977
Epoch 415/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0622 - accuracy: 0.9973
Epoch 416/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0616 - accuracy: 0.9977
Epoc

27/27 [==============================] - 0s 1ms/step - loss: 0.0286 - accuracy: 1.0000
Epoch 489/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0276 - accuracy: 1.0000
Epoch 490/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0272 - accuracy: 0.9996
Epoch 491/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0271 - accuracy: 0.9996
Epoch 492/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0269 - accuracy: 1.0000
Epoch 493/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0263 - accuracy: 0.9996
Epoch 494/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0260 - accuracy: 1.0000
Epoch 495/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0257 - accuracy: 1.0000
Epoch 496/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0253 - accuracy: 1.0000
Epoch 497/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0251 - accuracy: 1.0000
Epoc

In [236]:
estimator.predict(np.stack([training_set['X'].iloc[0]]))

array([[-1.]])

In [237]:
estimator.predict(np.stack([training_set['X'].iloc[-5]]))

array([[1.]])

In [238]:
# queries['query_use'] = queries['Query'].apply(use)

In [245]:
query_arr = use("what is a hypervisor?").numpy()[0]

corpus_for_query = corpus.copy()

def format_to_rank(training_set):
    raw_text_use = training_set['raw_text_use'].numpy()[0]
    concated = np.concatenate([query_arr, raw_text_use])
    return concated


corpus_for_query['use_with_query'] = corpus_for_query.apply(format_to_rank, axis=1)
corpus_for_query['is_match'] = estimator.predict(np.stack(corpus_for_query['use_with_query']))

for row in corpus_for_query[(corpus_for_query['is_match'] == 1.0)].to_dict(orient='record')[:10]:
    print('----------')
    if 'titleTag' in row:
        print(row['titleTag'])
    else:
        print(row['raw_text'][:100])
    print()
    print()
    print()
    print()

<ipython-input-245-0d75e17ae96a>:14: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



----------
Dynatrace for VMware Tanzu | Solutions Hub | VMware Tanzu




----------
Hazelcast IMDG Enterprise for VMware Tanzu | Solutions Hub | VMware Tanzu




----------
MPP BI | Greenplum Partner Marketplace  | VMware Tanzu




----------
T-Mobile | Customers  | VMware Tanzu




----------
Conversant | Customers  | VMware Tanzu




----------
Data Virtualization | Greenplum Partner Marketplace | VMware Tanzu




----------
Ops Manager | Tanzu Components  | VMware Tanzu




----------
Why Tanzu | VMware Tanzu




----------
Kroger | Customers  | VMware Tanzu




----------
VMware Tanzu Greenplum | VMware Tanzu






In [240]:
def write_submission():
    from time import time
    timestamp = str(time()).replace('.', '')
    fname = f'data/turnbull_submission_{timestamp}.csv'
    print("Writing To: ", fname)
    submission[['QueryId', 'DocumentId']].to_csv(fname, index=False)

In [71]:
submission[['QueryId', 'DocumentId']]

,QueryId,DocumentId
0,0,https---www.vmware.com-topics-glossary-content...
1,0,https---www.vmware.com-products-vsphere-hyperv...
2,0,https---www.vmware.com-topics-glossary-content...
3,0,https---www.vmware.com-topics-glossary-content...
4,0,https---www.vmware.com-topics-glossary-content...
...,...,...
11750,2350,https---www.vmware.com-topics-glossary-content...
11751,2350,en-2021-vRealize-Automation-8-3-Using-and-Mana...
11752,2350,en-2021-vRealize-Automation-8-4-Using-and-Mana...
11753,2350,en-2021-VMware-Cloud-Assembly-services-Using-a...


In [72]:
write_submission()

Writing To:  data/turnbull_submission_1652544680901428.csv


In [57]:
corpus[corpus['id'].str.contains('https---blogs.vmware.com-cloudprovider-2015-11-simplifying-cloud-spending-with-vmware-subscription-purchase-program.txt')]

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
11041,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Simplifying Cloud Spending with VMware Subscri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
